# Deliverables:

- Submit a single zip-compressed file that has the name: YourLastName_Exercise_1 that has the following files:

 1. Your **PDF document** that has your Source code and output
 2. Your **ipynb script** that has your Source code and output


# Objectives:

In this exercise, you will:

 - Construct hierarchical indexes
 - Select and group data to create pivot-tables




# Submission Formats :

Create a folder or directory with all supplementary files with your last name at the beginning of the folder name, compress that folder with zip compression, and post the zip-archived folder under the assignment link in Canvas. The following files should be included in an archive folder/directory that is uploaded as a single zip-compressed file. (Use zip, not StuffIt or any 7z or any other compression method.)


1. Complete IPYNB script that has the source code in Python used to access and analyze the data. The code should be submitted as an IPYNB script that can be be loaded and run in Jupyter Notebook for Python
2. Output from the program, such as console listing/logs, text files, and graphics output for visualizations. If you use the Data Science Computing Cluster or School of Professional Studies database servers or systems, include Linux logs of your sessions as plain text files. Linux logs may be generated by using the script process at the beginning of your session, as demonstrated in tutorial handouts for the DSCC servers.
3. List file names and descriptions of files in the zip-compressed folder/directory.


Formatting Python Code
When programming in Python, refer to Kenneth Reitz’ PEP 8: The Style Guide for Python Code:
http://pep8.org/ (Links to an external site.)Links to an external site.
There is the Google style guide for Python at
https://google.github.io/styleguide/pyguide.html (Links to an external site.)Links to an external site.
Comment often and in detail.


# Specifications and Requirements


We're going to use the XYZ data again  to construct hierarchical indexes and select, modify, group, and reshape data in a wide variety of ways.  The data we want here, which we'll call xyzcustnew, are as follows:

In [1]:

import pandas as pd  # panda's nickname is pd
import numpy as np  # numpy as np
from pandas import DataFrame, Series, Categorical 
from sqlalchemy import create_engine


In [2]:
engine=create_engine('sqlite:///xyz.db')    # the db is in my current working directory

In [3]:
xyzcustnew=pd.read_sql_table('xyzcust',engine)


In [4]:
# Refer to exercise #7 how we calculated this value for xyz db
heavyCut= 423  #heavyCut is a constant

In [5]:
heavyCat=Categorical(np.where(xyzcustnew.YTD_SALES_2009>heavyCut,1,0))
heavyCat.describe()

,counts,freqs
categories,,
0,25795,0.854733
1,4384,0.145267


In [6]:
heavyCat.rename_categories(['regular','heavy'],inplace=True)

In [7]:
heavyCat.describe()

,counts,freqs
categories,,
regular,25795,0.854733
heavy,4384,0.145267


In [8]:
heavyCat[:10]

['regular', 'heavy', 'regular', 'regular', 'regular', 'regular', 'heavy', 'regular', 'regular', 'regular']
Categories (2, object): ['regular', 'heavy']

In [9]:
xyzcustnew['heavyCat']=heavyCat

In [10]:
buyerType=pd.get_dummies(heavyCat)

In [11]:
buyerType[:3]

,regular,heavy
0,1,0
1,0,1
2,1,0


In [12]:
xyzcustnew['typeReg']=buyerType['regular']
xyzcustnew['typeHeavy']=buyerType['heavy']


In [13]:
xyzcustnew.columns

Index(['index', 'ACCTNO', 'ZIP', 'ZIP4', 'LTD_SALES', 'LTD_TRANSACTIONS',
       'YTD_SALES_2009', 'YTD_TRANSACTIONS_2009', 'CHANNEL_ACQUISITION',
       'BUYER_STATUS', 'ZIP9_SUPERCODE', 'heavyCat', 'typeReg', 'typeHeavy'],
      dtype='object')

In [14]:
# for this exercises we need to create trCountsChrono object similar to what we did in exercises #8

xyztrans=pd.read_sql('xyztrans', engine)

trandate=xyztrans.TRANDATE	# should be a Series

daystr=trandate.str[0:2]		# two digit date numbers slice

mostr=trandate.str[2:5]		# the three letter month abbreviations

yearstr=trandate.str[5:]		# four digit years


In [15]:
#create a dictionary for the months
monums={'JAN':'1', 'FEB':'2', 'MAR':'3', 'APR':'4', 'MAY':'5', 'JUN':'6', 'JUL':'7', 'AUG':'8', 'SEP':'9', 'OCT':'10', 'NOV':'11','DEC':'12'}
#month
monos=mostr.map(monums)	# do a dict lookup for each value of mostr

transtr=yearstr+'-'+monos+'-'+daystr

In [16]:
trDateTime=pd.to_datetime(transtr)

In [17]:

trCounts=trDateTime.value_counts()

In [18]:
newIndex=pd.date_range(trCounts.index.min(),trCounts.index.max())

trCountsChrono=trCounts.reindex(index=newIndex)


In [19]:
print(trCountsChrono.head())

2009-01-01    176
2009-01-02    305
2009-01-03    365
2009-01-04    231
2009-01-05    144
Freq: D, Name: TRANDATE, dtype: int64


In [20]:
trDF=DataFrame()                

In [21]:
trDF['date'] = trCountsChrono.index
trDF['transactions'] = trCountsChrono.values


In [22]:
trDF.columns

Index(['date', 'transactions'], dtype='object')

In [23]:
trDF.head()

,date,transactions
0,2009-01-01,176
1,2009-01-02,305
2,2009-01-03,365
3,2009-01-04,231
4,2009-01-05,144


In [24]:
trMed=trDF.transactions.median()		# here's the median

In [25]:
heavyLight=lambda x  : x >= trMed and 'heavy' or 'light'  # an example anon function

In [26]:
trDF['vol']=trDF.transactions.map(heavyLight)	# 'vol' is the heavy/light column

In [27]:
trDF['monum']=trDF.date.dt.month	# .dt is the datetime accessor

In [28]:
trDFnd=trDF.drop('date',axis=1)	# axis=1 means here a column is selected to drop

In [29]:
trDFgrouped=trDFnd.groupby(['monum','vol']).sum()

In [30]:
trDFgrouped.loc[11,'heavy']

transactions    8402
Name: (11, heavy), dtype: int64

In [31]:
 
trDFgrouped.loc[list(range(1,7))]

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [32]:
trDFgrouped.iloc[0:6]		# .iloc here, but .loc above.

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130
      light          1664

In [33]:
trDFgrouped[(3,'light'):(7,'heavy')]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440

In [34]:
trDFgrouped[(3,'light'):6]

transactions
monum vol                
3     light          1664
4     heavy          2327
      light          1727
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495

In [35]:
trDFgrouped.xs('light',level='vol')

,transactions
monum,
1,572
2,1625
3,1664
4,1727
5,2076
6,1495
7,564
8,1938
9,1942


In [36]:
trDFgrouped.xs('light',level='vol').T		# the transpose of the above

monum,1,2,3,4,5,6,7,8,9,10,11,12
transactions,572,1625,1664,1727,2076,1495,564,1938,1942,2241,49,257


In [37]:
mo=trDFgrouped.index.get_level_values(0) 	# the month numbers

In [38]:
volType=trDFgrouped.index.get_level_values(1)	# vol

In [39]:
trDFpiv=DataFrame({'month':mo,'vol': volType, 'transactions':trDFgrouped.transactions})		# data as a dict


In [40]:
trDFpived=trDFpiv.pivot(index='month',columns='vol',values='transactions')

In [41]:
trDFpiv['randy']=np.random.randn(len(trDFpiv))

In [42]:
trDFpived2=trDFpiv.pivot(index='month',columns='vol')

In [43]:
xyzdata=xyzcustnew[['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION']]

In [44]:
xyzgrouped=xyzdata.groupby(['BUYER_STATUS','heavyCat','CHANNEL_ACQUISITION'])


In [45]:
xyzCountData = xyzgrouped.size()	# a MultiIndexed Series of counts

In [46]:
print(xyzCountData.unstack())

CHANNEL_ACQUISITION     CB    IB    RT
BUYER_STATUS heavyCat                 
ACTIVE       regular   443  1112  7393
             heavy     356   703  3325
INACTIVE     regular   691  1249  7056
             heavy       0     0     0
LAPSED       regular   372  1111  6368
             heavy       0     0     0


In [47]:
unStackxyz=xyzCountData.unstack()		# what we had just above

In [48]:
unStackxyz.T.stack()		# .T is the transpose 

BUYER_STATUS                  ACTIVE  INACTIVE  LAPSED
CHANNEL_ACQUISITION heavyCat                          
CB                  regular      443       691     372
                    heavy        356         0       0
IB                  regular     1112      1249    1111
                    heavy        703         0       0
RT                  regular     7393      7056    6368
                    heavy       3325         0       0

In [49]:
unStackxyz.T.stack(0).unstack(1)

heavyCat            regular                  heavy                
BUYER_STATUS         ACTIVE INACTIVE LAPSED ACTIVE INACTIVE LAPSED
CHANNEL_ACQUISITION                                               
CB                      443      691    372    356        0      0
IB                     1112     1249   1111    703        0      0
RT                     7393     7056   6368   3325        0      0

In [50]:
unStackxyz.T.stack(level=['heavyCat','BUYER_STATUS'])

CHANNEL_ACQUISITION  heavyCat  BUYER_STATUS
CB                   regular   ACTIVE           443
                               INACTIVE         691
                               LAPSED           372
                     heavy     ACTIVE           356
                               INACTIVE           0
                               LAPSED             0
IB                   regular   ACTIVE          1112
                               INACTIVE        1249
                               LAPSED          1111
                     heavy     ACTIVE           703
                               INACTIVE           0
                               LAPSED             0
RT                   regular   ACTIVE          7393
                               INACTIVE        7056
                               LAPSED          6368
                     heavy     ACTIVE          3325
                               INACTIVE           0
                               LAPSED             0
dtype: int64

In [51]:
unStackxyz.T.stack(level=['BUYER_STATUS','heavyCat'])

CHANNEL_ACQUISITION  BUYER_STATUS  heavyCat
CB                   ACTIVE        regular      443
                                   heavy        356
                     INACTIVE      regular      691
                                   heavy          0
                     LAPSED        regular      372
                                   heavy          0
IB                   ACTIVE        regular     1112
                                   heavy        703
                     INACTIVE      regular     1249
                                   heavy          0
                     LAPSED        regular     1111
                                   heavy          0
RT                   ACTIVE        regular     7393
                                   heavy       3325
                     INACTIVE      regular     7056
                                   heavy          0
                     LAPSED        regular     6368
                                   heavy          0
dtype: int64

In [52]:
xyzcust=xyzcustnew[['BUYER_STATUS','heavyCat','LTD_SALES']].copy()

In [53]:
xyzcustm=pd.melt(xyzcust,id_vars=['BUYER_STATUS','heavyCat'],var_name="LTD_SALES")


In [54]:
print(xyzcustm)

      BUYER_STATUS heavyCat  LTD_SALES   value
0         INACTIVE  regular  LTD_SALES    90.0
1           ACTIVE    heavy  LTD_SALES  4227.0
2           ACTIVE  regular  LTD_SALES   420.0
3         INACTIVE  regular  LTD_SALES  6552.0
4           ACTIVE  regular  LTD_SALES   189.0
...            ...      ...        ...     ...
30174       ACTIVE  regular  LTD_SALES  2736.0
30175       ACTIVE  regular  LTD_SALES  2412.0
30176     INACTIVE  regular  LTD_SALES   429.0
30177     INACTIVE  regular  LTD_SALES   651.0
30178       ACTIVE    heavy  LTD_SALES  4527.0

[30179 rows x 4 columns]


In [55]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS','heavyCat'],columns=['CHANNEL_ACQUISITION'])


CHANNEL_ACQUISITION             CB           IB           RT
BUYER_STATUS heavyCat                                       
ACTIVE       regular    205.334086   191.047662   167.993913
             heavy     2397.606742  1251.559033  1158.506165
INACTIVE     regular      0.000000     0.000000     0.000000
LAPSED       regular      0.000000     0.000000     0.000000

In [56]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'])


heavyCat                regular                                heavy  \
CHANNEL_ACQUISITION          CB          IB          RT           CB   
BUYER_STATUS                                                           
ACTIVE               205.334086  191.047662  167.993913  2397.606742   
INACTIVE               0.000000    0.000000    0.000000          NaN   
LAPSED                 0.000000    0.000000    0.000000          NaN   

heavyCat                                       
CHANNEL_ACQUISITION           IB           RT  
BUYER_STATUS                                   
ACTIVE               1251.559033  1158.506165  
INACTIVE                     NaN          NaN  
LAPSED                       NaN          NaN

In [57]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   

heavyCat                        
CHANNEL_ACQUISITION         RT  
BUYER_STATUS                    
ACTIVE               3852033.0  
INACTIVE                   NaN  
LAPSED                     NaN

In [58]:
pd.pivot_table(xyzcustnew,values='YTD_SALES_2009',index=['BUYER_STATUS'],columns=['heavyCat','CHANNEL_ACQUISITION'],aggfunc=np.sum,margins=True)


heavyCat             regular                          heavy            \
CHANNEL_ACQUISITION       CB        IB         RT        CB        IB   
BUYER_STATUS                                                            
ACTIVE               90963.0  212445.0  1241979.0  853548.0  879846.0   
INACTIVE                 0.0       0.0        0.0       NaN       NaN   
LAPSED                   0.0       0.0        0.0       NaN       NaN   
All                  90963.0  212445.0  1241979.0  853548.0  879846.0   

heavyCat                              All  
CHANNEL_ACQUISITION         RT             
BUYER_STATUS                               
ACTIVE               3852033.0  7130814.0  
INACTIVE                   NaN        0.0  
LAPSED                     NaN        0.0  
All                  3852033.0  7130814.0

In [59]:
xyzGrouper=xyzcustnew.groupby(['BUYER_STATUS','heavyCat'])

In [60]:
xyzGrouper.agg({'YTD_SALES_2009': [np.mean, np.std],'LTD_SALES':[np.mean,np.std]})


YTD_SALES_2009                 LTD_SALES              
                                mean          std         mean           std
BUYER_STATUS heavyCat                                                       
ACTIVE       regular      172.707532   107.584023  1001.845105   1466.075631
             heavy       1274.048130  5434.616517  4096.179745  34210.646330
INACTIVE     regular        0.000000     0.000000   568.014784    850.966479
             heavy               NaN          NaN          NaN           NaN
LAPSED       regular        0.000000     0.000000   841.467329   1374.447756
             heavy               NaN          NaN          NaN           NaN

In [61]:
def coefV(x):                       # a baby CV function that accepts a sequence
    return np.std(x)/np.mean(x)


In [62]:
buyerStats=xyzcustnew[['BUYER_STATUS','LTD_SALES','LTD_TRANSACTIONS']]
buyerGrouper=buyerStats.groupby(['BUYER_STATUS'])
buyerGrouper.agg(coefV)

,LTD_SALES,LTD_TRANSACTIONS
BUYER_STATUS,,
ACTIVE,9.758480,1.153501
INACTIVE,1.498058,0.784441
LAPSED,1.633290,0.987139


In [63]:
def ptiles(x):
    p5=np.percentile(x,5)
    p95=np.percentile(x,95)
    return p5, p95


In [64]:
buyerGrouper.agg([np.mean, ptiles])

LTD_SALES                            LTD_TRANSACTIONS  \
                     mean                     ptiles             mean   
BUYER_STATUS                                                            
ACTIVE        2019.364086  (81.0, 6544.349999999997)         6.935794   
INACTIVE       568.014784             (60.0, 1776.0)         2.263895   
LAPSED         841.467329             (63.0, 2904.0)         3.498280   

                           
                   ptiles  
BUYER_STATUS               
ACTIVE        (1.0, 20.0)  
INACTIVE       (1.0, 6.0)  
LAPSED         (1.0, 9.0)

In [65]:
buyerGrouper.agg([np.mean,ptiles]).loc['ACTIVE','LTD_SALES']

mean                        2019.36
ptiles    (81.0, 6544.349999999997)
Name: ACTIVE, dtype: object

# Requirements :
1. Get the trDFgrouped data starting from the May heavy day counts to the August heavy counts
2. Group xyz customers using BUYER_STATUS, heavyCat, and ZIP, and apply np.sum function on the aggregated data for YTD_SALES_2009 and LTD_SALES columns

In [81]:
# Write your python code that meets the above requirements in this cell
trDFgrouped.head()

transactions
monum vol                
1     heavy          5255
      light           572
2     heavy           761
      light          1625
3     heavy          1130

In [82]:
xyzcustnew.head()

,index,ACCTNO,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,CHANNEL_ACQUISITION,BUYER_STATUS,ZIP9_SUPERCODE,heavyCat,typeReg,typeHeavy
0,0,WDQQLLDQL,60084,5016,90.0,1,0.0,0,IB,INACTIVE,600845016,regular,1,0
1,1,WQWAYHYLA,60091,1750,4227.0,9,1263.0,3,RT,ACTIVE,600911750,heavy,0,1
2,2,GSHAPLHAW,60067,900,420.0,3,129.0,1,RT,ACTIVE,600670900,regular,1,0
3,3,PGGYDYWAD,60068,3838,6552.0,6,0.0,0,RT,INACTIVE,600683838,regular,1,0
4,4,LWPSGPLLS,60090,3932,189.0,3,72.0,1,RT,ACTIVE,600903932,regular,1,0


In [83]:
trDFgrouped[(5, 'heavy'):(8, 'heavy')]

transactions
monum vol                
5     heavy          2172
      light          2076
6     heavy          2878
      light          1495
7     heavy          4440
      light           564
8     heavy          1682

In [84]:
xyzgrouped=xyzcustnew.groupby(['BUYER_STATUS', 'heavyCat', 'ZIP'])
xyzgrouped

In [85]:
xyzcustnew.groupby('BUYER_STATUS').sum()

,index,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,ZIP9_SUPERCODE,typeReg,typeHeavy
BUYER_STATUS,,,,,,,,,,
ACTIVE,203477100,800940182,48106006,26922162.0,92468,7130814.0,25494,7975212407853,8948.0,4384.0
INACTIVE,136284338,540437361,27204108,5109861.0,20366,0.0,0,5376776549876,8996.0,0.0
LAPSED,120111048,471596228,24873867,6606360.0,27465,0.0,0,4698569597495,7851.0,0.0


In [86]:
xyzcustnew.groupby('heavyCat').sum()

,index,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,ZIP9_SUPERCODE,typeReg,typeHeavy
heavyCat,,,,,,,,,,
regular,393456475,1549592938,84380896,20680731.0,93540,1545387.0,11861,15426945100715,25795.0,0.0
heavy,66416011,263380833,15803085,17957652.0,46759,5585427.0,13633,2623613454509,0.0,4384.0


In [87]:
xyzcustnew.groupby('ZIP').sum()

,index,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,ZIP9_SUPERCODE,typeReg,typeHeavy
ZIP,,,,,,,,,
0,19524,0,111.0,1,0.0,0,600933127,1.0,0.0
60056,23159002,3813813,1360560.0,6444,246858.0,1090,908051453635,1351.0,164.0
60060,19280456,4394210,1355868.0,5477,282294.0,1071,766370946361,1105.0,178.0
60061,18066932,2971076,1301277.0,5537,221373.0,1019,716531245397,1046.0,151.0
60062,45922551,12396436,3378210.0,12726,649488.0,2314,1819293681415,2633.0,426.0
60064,716835,85632,33156.0,171,9882.0,44,23425183588,35.0,6.0
60065,224243,24881,16866.0,65,1536.0,8,12613676887,20.0,1.0
60067,45932874,13976687,3891255.0,16228,700473.0,2847,1810435697532,2564.0,458.0
60068,41227114,8684839,2947179.0,12028,520986.0,2016,1651279477837,2418.0,336.0


#### Group xyz customers using BUYER_STATUS, heavyCat, and ZIP, and apply np.sum function on the aggregated data for YTD_SALES_2009 and LTD_SALES columns

In [88]:
xyzcustnew.groupby('YTD_SALES_2009').sum()

,index,ZIP,ZIP4,LTD_SALES,LTD_TRANSACTIONS,YTD_TRANSACTIONS_2009,ZIP9_SUPERCODE,typeReg,typeHeavy
YTD_SALES_2009,,,,,,,,,
0.0,256395386,1012033589,52077975,11716221.0,47831,0,10075346147371,16847.0,0.0
3.0,17134,60061,1242,1995.0,5,1,600611242,1.0,0.0
6.0,142262,540651,32091,5748.0,39,9,5406551095,9.0,0.0
9.0,160272,540673,29653,6822.0,27,9,5406763105,9.0,0.0
12.0,304233,901114,48482,5967.0,74,15,9011188482,15.0,0.0
...,...,...,...,...,...,...,...,...,...
14448.0,139,60093,3748,14448.0,5,5,600933748,0.0,1.0
14817.0,28822,60062,7141,16977.0,5,4,600627141,0.0,1.0
15273.0,2047,60093,3708,23910.0,8,4,600933708,0.0,1.0


In [89]:
xyzcustnew.groupby('LTD_SALES').sum()

,index,ZIP,ZIP4,LTD_TRANSACTIONS,YTD_SALES_2009,YTD_TRANSACTIONS_2009,ZIP9_SUPERCODE,typeReg,typeHeavy
LTD_SALES,,,,,,,,,
3.0,31929,120160,3974,2,0.0,0,1201603974,2,0
6.0,139107,420511,18182,7,18.0,3,4205137186,7,0
9.0,131353,420505,29723,7,36.0,4,4205079723,7,0
12.0,234891,720843,44352,12,48.0,4,7208474352,12,0
15.0,292852,1321663,59060,22,120.0,8,13216704090,22,0
...,...,...,...,...,...,...,...,...,...
47307.0,1510,60067,6613,48,12684.0,9,600676613,0,1
49302.0,9544,60093,1563,123,9900.0,26,600931563,0,1
65643.0,19776,60068,2650,27,9453.0,5,600682650,0,1


In [91]:
xyzcustnew.groupby('YTD_SALES_2009')['LTD_SALES'].sum()

YTD_SALES_2009
0.0         11716221.0
3.0             1995.0
6.0             5748.0
9.0             6822.0
12.0            5967.0
               ...    
14448.0        14448.0
14817.0        16977.0
15273.0        23910.0
28002.0       122352.0
351000.0     2247750.0
Name: LTD_SALES, Length: 1068, dtype: float64